In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from preprocess import preprocess_text
from features.vectorizer import Vectorizer
from utils.constants import CATEGORIES, TITLE, ABSTRACT, TEXT_FEATURE
from models.supervised_model import SupervisedModel
from sklearn.neighbors import KNeighborsClassifier
import warnings
from tqdm import tqdm
import pickle
warnings.filterwarnings("ignore") 


In [39]:
with open("features/count_vectorizer.pkl",'rb') as f:
    cv = pickle.load(f)

In [40]:
cv

,line,algorithm,relationship,random,paper,introduce,markov,model,linear,space,...,score,baseline,android,caching,lte,apps,iot,sdn,blockchain,categories
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40490,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40491,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
40492,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
40493,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [42]:
y = cv['categories']


In [43]:
X = cv.drop(columns=['categories'])

In [47]:
record_label = 6
minority_labels = cv[cv['categories']==record_label]
minority_labels = minority_labels.reset_index(drop=True)

def get_knn_matrix():
    
    K_range = [5,10,15,20]
    result = pd.DataFrame()
    for k in K_range:
        neigh = KNeighborsClassifier(n_neighbors=k)
        neigh.fit(X, y)
        x = None
        ir = []
        for index in tqdm(range(len(minority_labels))):
            record = minority_labels.loc[index, :].reset_index().T
            record = record[record.columns[:-1]]
            record = record.drop('index')
            x = record
            n = neigh.kneighbors(x)
            labels = cv.loc[n[1].tolist()[0]].reset_index(drop=True)
            ir.append(len(labels[labels['categories'] == record_label]) / len(labels))
        result["k"+str(k)] = ir
        result.to_csv("sampling_result.csv", index=False)
    return result

In [48]:
result = get_knn_matrix()
result.to_pickle("sampling_final_result.pkl")

100%|██████████| 1286/1286 [20:53<00:00,  1.03it/s]


In [49]:
result

,k5,k10,k15,k20
0,0.4,0.4,0.400000,0.40
1,0.6,0.5,0.400000,0.30
2,0.2,0.1,0.066667,0.05
3,0.4,0.4,0.266667,0.20
4,0.6,0.3,0.200000,0.15
...,...,...,...,...
1281,0.6,0.3,0.200000,0.15
1282,0.6,0.4,0.266667,0.20
1283,0.8,0.4,0.333333,0.25
1284,0.8,0.5,0.466667,0.35


In [50]:
result.mean(axis=0)

k5     0.476672
k10    0.364852
k15    0.317678
k20    0.286703
dtype: float64